In [1]:
import requests
# from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import pickle
import time

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from collections import defaultdict

import pipeline as p

%load_ext autoreload
%autoreload 2

In [2]:
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"

pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)

chromedriver = "/Users/kendra/ds/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url_base = 'https://www.pro-football-reference.com'

team_dict = p.open_pkl('Data/team_dictionary.pkl')

Create a dictionary to track which teams & seasons we've scraped data for

In [11]:
# season_dict:
    # key = [year] 
    # vals = list of teams whose team-season page has been sucessfully scraped

# season_dict = p.open_pkl('season_dict.pkl')    # use this if need to re-start scraping
season_dict = defaultdict(list)  # [year] = list of teams whose team-season page has been sucessfully scraped

### Iterate over team-season pages, and boxscores within a given season:

In [12]:
driver = webdriver.Chrome(chromedriver, desired_capabilities=caps)

for year in range(2018, 2019):
    print(year)
    count = 0
    bxsc_html = defaultdict(list)    # [href] = html .also tracks whether a bxsc href has been visited
    bxsc_refs = {}                   # [team-year] = list of hrefs for boxscores
    season_html = {}                 # [team-year] = html of team season page
    
    for tm_list in team_dict.values():
        team, url = tm_list[1], tm_list[2]
        tm_yr = team +'-' + str(year)
        if team not in season_dict[year]:

            # get html of season data table
            html = p.get_season_data(url, year, count, driver)

            # if successfully returned data:
            if html:
                # add html to season_html dictionary
                season_html[tm_yr] = html

                # generate list of individaul game boxscore URLs, add them to bxsc_refs dictionary
                href_list = p.get_href_list(html)
                bxsc_refs[tm_yr] = href_list

                # get & add bxsc html's to bxsc_html dict
                for a in href_list:
                    href = a.attrs['href']
                    if href not in bxsc_html.keys():
                        bxsc_html[href] = p.get_bxsc_data(href, driver)
                    else:
                        print('Already scraped data this boxscore')

                # add team & year to season dictionary
                season_dict[year].append(team)

            # increment counter
            count += 1
            print(count)

#     p.pkl_this('bxsc_refs_' + str(year) + '.pkl',bxsc_refs)
    p.save_dict_txt('bxsc_refs_' + str(year) + '.txt',bxsc_refs)
    p.pkl_this('bxsc_html_' + str(year) + '.pkl', bxsc_html)   # changed this to include year after I ran
    p.pkl_this('season_html_' + str(year) + '.pkl', season_html)
    p.pkl_this('season_dict.pkl', season_dict)
#     p.log_this(str(year) + ': ' + str(len(bxsc_html) + ', ' + str(len(bxsc_refs)) + ', ' len(season_dict[year]), len(season_html))
    
p.close_dr()

2018
Loading page for tam 2018
Page loaded & data found!
Skipping page https://www.pro-football-reference.com/boxscores/201810140atl.htm
Skipping page https://www.pro-football-reference.com/boxscores/201810210tam.htm
Skipping page https://www.pro-football-reference.com/boxscores/201810280cin.htm
Skipping page https://www.pro-football-reference.com/boxscores/201811040car.htm
Skipping page https://www.pro-football-reference.com/boxscores/201811110tam.htm
Skipping page https://www.pro-football-reference.com/boxscores/201811180nyg.htm
Skipping page https://www.pro-football-reference.com/boxscores/201811250tam.htm
Skipping page https://www.pro-football-reference.com/boxscores/201812020tam.htm
Skipping page https://www.pro-football-reference.com/boxscores/201812090tam.htm
Skipping page https://www.pro-football-reference.com/boxscores/201812160rav.htm
Skipping page https://www.pro-football-reference.com/boxscores/201812230dal.htm
Skipping page https://www.pro-football-reference.com/boxscores/

Already scraped data this boxscore
Skipping page https://www.pro-football-reference.com/boxscores/201810140htx.htm
Already scraped data this boxscore
Skipping page https://www.pro-football-reference.com/boxscores/201810250htx.htm
Already scraped data this boxscore
Skipping page https://www.pro-football-reference.com/boxscores/201811180was.htm
Already scraped data this boxscore
Skipping page https://www.pro-football-reference.com/boxscores/201812020htx.htm
Skipping page https://www.pro-football-reference.com/boxscores/201812090htx.htm
Skipping page https://www.pro-football-reference.com/boxscores/201812150nyj.htm
Skipping page https://www.pro-football-reference.com/boxscores/201812230phi.htm
Already scraped data this boxscore
10
Loading page for kan 2018
Page loaded & data found!
Already scraped data this boxscore
Already scraped data this boxscore
Already scraped data this boxscore
Already scraped data this boxscore
Skipping page https://www.pro-football-reference.com/boxscores/2018102

Skipping page https://www.pro-football-reference.com/boxscores/201811040mia.htm
Skipping page https://www.pro-football-reference.com/boxscores/201811110gnb.htm
Skipping page https://www.pro-football-reference.com/boxscores/201811250clt.htm
Skipping page https://www.pro-football-reference.com/boxscores/201812020mia.htm
Already scraped data this boxscore
Already scraped data this boxscore
Already scraped data this boxscore
Skipping page https://www.pro-football-reference.com/boxscores/201812300buf.htm
20
Loading page for phi 2018
Page loaded & data found!
Already scraped data this boxscore
Already scraped data this boxscore
Already scraped data this boxscore
Already scraped data this boxscore
Skipping page https://www.pro-football-reference.com/boxscores/201810210phi.htm
Already scraped data this boxscore
Already scraped data this boxscore
Skipping page https://www.pro-football-reference.com/boxscores/201811180nor.htm
Skipping page https://www.pro-football-reference.com/boxscores/2018112

In [6]:
p.close_dr()

Already closed
